#### Ethan Zheng, 05/08/2019

### Background and introduction
Every syllable in English must have a vowel (a monophthong or a diphthong) as its nucleus. An interesting problem would be to count how many syllables a word has, and group the phonemes into their corresponding syllables. The easier version of this problem is when we are given the IPA expression of the word, where we can accurately examine the phonemes. The harder problem is when we are given the orthographical form of a word, can we still use a program to count the syllables accountably. If this is made possible, can we separate the syllables and match them to their corresponding IPA snippets?

### A first problem: Getting a dictionary of words with their corresponding IPA forms
NLTK has provided a corpus that fits this requirement very well.

In [1]:
import nltk

dictionary = nltk.corpus.cmudict.dict()
print("Dictionary size: ", end = "")
print(len(dictionary))

wordList = ['fastidious', 'liability', 'automatic', 
            'thanks', 'trace', 'finish', 'decline', 
            'forbid', 'injection', 'difficulty'
           ]
#             , 'craftsman', 'replacement', 'presidential', 
#             'virgin', 'advantage', 'complain', 'investment',
#             'bay', 'tear', 'orientation', 'file', 'slot', 
#             'barrel', 'contraction', 'encourage', 'multiply', 
#             'variety', 'variation', 'likely', 'complication', 
#             'opera', 'behave', 'economy', 'dish', 'photograph',
#             'rotation', 'stimulation', 'bottom', 'curve', 
#             'minority', 'way', 'innovation', 'exaggerate', 
#             'thesis', 'hardship', 'mechanism', 'lamp', 'salesperson', 
#             'inflate', 'page', 'dog', 'rank', 'strict', 'physical',
#             'space', 'factory', 'prison','enthusiasm', 'flash', 
#             'session', 'smash', 'communication', 'stick', 'degree', 
#             'question', 'trust', 'photocopy', 'canvas', 'fun', 
#             'cool', 'slide', 'penalty', 'corn', 'deliver', 
#             'intensify', 'neighbor', 'hope', 'exception', 'mayor', 
#             'forget', 'neck', 'afford', 'mutter', 'great', 
#             'forward', 'tablet', 'glasses', 'bed', 'lazy', 
#             'receipt', 'reach', 'lawyer', 'commemorate', 'elephant',
#             'advertising', 'dialogue', 'food', 'feather', 
#             'treasurer', 'chaos']

# Randomly generated wordList from https://randomwordgenerator.com/

Dictionary size: 123455


### Count syllables by vowels
Since each syllable is marked by a vowel as its kernel, the number if syllables is the number of vowels. In the list of phonemes the dictionary gives us, consonants are one or two characters long, and vowels are three characters long with the last one being the level of stress. This gives us a very handy way to sort out vowels from consonants.

In [2]:
def countSyllables(phonemeList):
    count = 0
    for phoneme in phonemeList:
        if(len(phoneme) == 3):
            count += 1
    return count

for word in wordList:
    phonemes = dictionary[word][0]
    if(len(word) <= 7):
        print(word, end = '\t\t')
    else:
        print(word, end = '\t')
    print("Syllables: ", end = "")
    print(countSyllables(phonemes), end = '\t')
    print(phonemes)

fastidious	Syllables: 4	['F', 'AE0', 'S', 'T', 'IH1', 'D', 'IY0', 'AH0', 'S']
liability	Syllables: 5	['L', 'AY2', 'AH0', 'B', 'IH1', 'L', 'IH0', 'T', 'IY0']
automatic	Syllables: 4	['AO2', 'T', 'AH0', 'M', 'AE1', 'T', 'IH0', 'K']
thanks		Syllables: 1	['TH', 'AE1', 'NG', 'K', 'S']
trace		Syllables: 1	['T', 'R', 'EY1', 'S']
finish		Syllables: 2	['F', 'IH1', 'N', 'IH0', 'SH']
decline		Syllables: 2	['D', 'IH0', 'K', 'L', 'AY1', 'N']
forbid		Syllables: 2	['F', 'ER0', 'B', 'IH1', 'D']
injection	Syllables: 3	['IH0', 'N', 'JH', 'EH1', 'K', 'SH', 'AH0', 'N']
difficulty	Syllables: 4	['D', 'IH1', 'F', 'AH0', 'K', 'AH0', 'L', 'T', 'IY0']


### Group phonemes into their corresponding syllables

#### General Rules (updated):
1. Each syllable has a vowel as nucleus.
2. If a vowel is followed by another vowel, the second vowel forms an onsetless syllable.
3. If a vowel appears at the word initial, it forms an onsetless syllable.
4. The consonant cluster that appears before a vowel is the onset of that syllable. According to maximal onset principle, we always want to maximize the length of an onset given that it's possible.
5. After we get the onset, the remaining of the consonant cluster is the coda of the previous syllable.

#### Maximal onset principle:
1. strike /stɹaɪk/: The onset is /stɹ/.
2. swim /swɪm/: The onset is /sw/.
3. construct /ˈkɑn.stɹʌkt/: The onset of the second syllable is /stɹ/ because /nstɹ/ is impossible.
4. eyebrow /ˈaɪˌbɹaʊ/: The onset of the second syllable is /br/ because it is allowed in English.
5. ghastly /ˈɡæs(t).li/: The onset of the second syllable is /l/ because /tl/ and /stl/ are impossible. 

#### Possible consonant clusters for onsets in English:
(Data acquired from http://usefulenglish.ru/phonetics/practice-consonant-clusters, with modification)
1. /pl/, /pɹ/, /bl/, /bɹ/,
2. /tɹ/, /dɹ/,
3. /kl/, /kɹ/, /gl/, /gɹ/,
4. /fl/, /fɹ/,
5. /ʃɹ/, /θɹ/,
6. /sk/, /skɹ/,
7. /sl/, /sm/, /sn/,
8. /sp/, /spl/, /spɹ/,
9. /st/, /stɹ/,
10. /sw/, /tw/, /dw/, /kw/, /skw/, /gw/

#### Taking these into account, I have a program designed as such:

In [3]:
onsetClusters = [
    ['P','L'], ['P','R'], ['B','L'], ['B','R'], 
    ['T','R'], ['D','R'], 
    ['K','L'], ['K','R'], ['G','L'], ['G','R'], 
    ['F','L'], ['F','R'], 
    ['SH','R'], ['TH','R'], 
    ['S','K'], ['S','K','R'], 
    ['G','L'], ['G','R'], 
    ['S','L'], ['S','M'], ['S','N'], 
    ['S','P'], ['S','P','L'], ['S','P','R'], 
    ['S','T'], ['S','T','R'], 
    ['S','W'], ['T','W'], ['D','W'], ['K','W'], ['S','K','W'], ['G','W']
]

def listEquals(l1, l2):
    if(len(l1) != len(l2)):
        return False
    for i in range(len(l1)):
        if(l1[i] != l2[i]):
            return False
    return True

def isValidOnset(cluster):
    for valid in onsetClusters:
        if(listEquals(cluster, valid)):
            return True
    return False

def groupPhonemes(phonemeList):
    grouped = []
    temp = []
    prevPhoneme = ''
    isWordInitial = True
    for phoneme in phonemeList:
        if len(phoneme) == 3:
            if not isWordInitial:
                if len(prevPhoneme) != 3:
                    cluster = temp[-3:]
                    if(isValidOnset(cluster)):
                        del temp[-3:]
                    else:
                        cluster = temp[-2:]
                        if(isValidOnset(cluster)):
                            del temp[-2:]
                        else:
                            cluster = temp[-1:]
                            del temp[-1:]
                    grouped.append(temp)
                    temp = cluster
                else:
                    grouped.append(temp)
                    temp = []
            else:
                isWordInitial = False
        temp.append(phoneme)
        prevPhoneme = phoneme
    grouped.append(temp)
    return grouped
    
exampleList = wordList + ['strike','swim','construct','eyebrow','ghastly']

for word in exampleList:
    phonemes = dictionary[word][0]
    if(len(word) <= 7):
        print(word, end = '\t\t')
    else:
        print(word, end = '\t')
    print(groupPhonemes(phonemes))

fastidious	[['F', 'AE0'], ['S', 'T', 'IH1'], ['D', 'IY0'], ['AH0', 'S']]
liability	[['L', 'AY2'], ['AH0'], ['B', 'IH1'], ['L', 'IH0'], ['T', 'IY0']]
automatic	[['AO2'], ['T', 'AH0'], ['M', 'AE1'], ['T', 'IH0', 'K']]
thanks		[['TH', 'AE1', 'NG', 'K', 'S']]
trace		[['T', 'R', 'EY1', 'S']]
finish		[['F', 'IH1'], ['N', 'IH0', 'SH']]
decline		[['D', 'IH0'], ['K', 'L', 'AY1', 'N']]
forbid		[['F', 'ER0'], ['B', 'IH1', 'D']]
injection	[['IH0', 'N'], ['JH', 'EH1', 'K'], ['SH', 'AH0', 'N']]
difficulty	[['D', 'IH1'], ['F', 'AH0'], ['K', 'AH0', 'L'], ['T', 'IY0']]
strike		[['S', 'T', 'R', 'AY1', 'K']]
swim		[['S', 'W', 'IH1', 'M']]
construct	[['K', 'AH0', 'N'], ['S', 'T', 'R', 'AH1', 'K', 'T']]
eyebrow		[['AY1'], ['B', 'R', 'AW2']]
ghastly		[['G', 'AE1', 'S', 'T'], ['L', 'IY0']]


### Verification

This block of code compares the syllables returned by direct count (by the number of vowels) and the number of phoneme groups in the list of syllables returned by the grouping function.

As you can see, the grouping function gives the correct number of syllables.

In [4]:
for word in exampleList:
    phonemes = dictionary[word][0]
    if(len(word) <= 7):
        print(word, end = '\t\t')
    else:
        print(word, end = '\t')
    syllableCount = countSyllables(phonemes)
    grouped = groupPhonemes(phonemes)
    print('Direct Count: ', end = '')
    print(syllableCount, end = '\t\tGroup Count: ')
    print(len(grouped), end = '\t\tMatch: ')
    print(len(grouped) == syllableCount)

fastidious	Direct Count: 4		Group Count: 4		Match: True
liability	Direct Count: 5		Group Count: 5		Match: True
automatic	Direct Count: 4		Group Count: 4		Match: True
thanks		Direct Count: 1		Group Count: 1		Match: True
trace		Direct Count: 1		Group Count: 1		Match: True
finish		Direct Count: 2		Group Count: 2		Match: True
decline		Direct Count: 2		Group Count: 2		Match: True
forbid		Direct Count: 2		Group Count: 2		Match: True
injection	Direct Count: 3		Group Count: 3		Match: True
difficulty	Direct Count: 4		Group Count: 4		Match: True
strike		Direct Count: 1		Group Count: 1		Match: True
swim		Direct Count: 1		Group Count: 1		Match: True
construct	Direct Count: 2		Group Count: 2		Match: True
eyebrow		Direct Count: 2		Group Count: 2		Match: True
ghastly		Direct Count: 2		Group Count: 2		Match: True


### The next problem: Matching the grouped syllables to orthographical form

In order to do this, we need to create a dictionary that maps spellings fragments to phonemes. We first make a copy of our corpus and delete the stress marking for the sake of convenience. Then we make a set of all phonemes that the corpus contains.

In [5]:
import copy

dictionaryCopy = copy.deepcopy(dictionary)
phonemeSet = set()

for key, values in dictionaryCopy.items():
    for possibleValue in values:
        for phoneme in possibleValue:
            if len(phoneme) == 3:
                phoneme = phoneme[:2]
            phonemeSet.add(phoneme)

print(sorted(phonemeSet))
print("Elements count: ", end = "")
print(len(phonemeSet))

['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH']
Elements count: 39


Now we know what phonemes there are. So we can start to make a simple dictionary that maps phonemes to spelling.

In [13]:
phonemeDict = {
    'AA': ['oo','ou','u','o'], 
    'AE': ['ai','a'], 
    'AH': ['eur','our','ur','re','or','oo','ou','er','ar','a','u','o','i','e',''],
    'AO': ['au','ho','o','a'], 
    'AW': ['ough','ow','ou'], 
    'AY': ['eigh','eye','igh','ie','uy','ye','ai','is','i','y'], 
    'B' : ['bb','b'], 
    'CH': ['tch','ch','tu','ti','te'], 
    'D' : ['dd','ed','d'], 
    'DH': ['th'], 
    'EH': ['eo','ei','ae','ay','ie','ai','ea','e','u','a'], 
    'ER': ['eur','our','er','ar','ir','or','ur','re'], 
    'EY': ['eigh','ea','ay','ai','a'], 
    'F' : ['ff','ph','gh','lf','ft','f'], 
    'G' : ['gue','gg','gh','gu','g'], 
    'HH': ['wh','h'], 
    'IH': ['ie','ui','a','i','e','o','u','y'], 
    'IY': ['ee','ea','ey','oe','ie','ei','eo','ay','e','i','y'], 
    'JH': ['dge','di','gg','ge','g','j'], 
    'K' : ['que','ch','cc','ke','lk','qu','ck','q','k','c','x'], 
    'L' : ['ll','le','l'], 
    'M' : ['mm','mb','mn','lm','m'], 
    'N' : ['nn','kn','gn','pn','ne','n'], 
    'NG': ['ngue','ng','n'], 
    'OW': ['ough','eau','oa','oe','ow','oo','eu','o'], 
    'OY': ['ouy','oi','oy'], 
    'P' : ['pp','p'], 
    'R' : ['rr','wr','rh','r'], 
    'S' : ['ss','sc','ps','st','ce','se','c','s'], 
    'SH': ['sci','sh','ce','ci','si','ch','ti','s'], 
    'T' : ['tt','th','ed','t'], 
    'TH': ['th'], 
    'UH': [], 
    'UW': ['ough','oeu','oo','oe','ew','ue','ui','ou','o','u'], 
    'V' : ['ph','ve','v','f'], 
    'W' : ['wh','w','u','o'], 
    'Y' : ['y','i','j',''], 
    'Z' : ['zz','ss','ze','se','z','s','x'], 
    'ZH': ['si','s','z']
}

# Data modified from https://www.dyslexia-reading-well.com/support-files
#/the-44-phonemes-of-english.pdf

The idea here is that with a list of phoneme for the word, we iterate through all the phonemes in the list and try to fit a spelling to it. For each phoneme, we pick the first one available in the list of spelling it corresponds to, and try to fit it. If the choice is correct, we proceed on to the next phoneme. Otherwise, when it doesn't have any spelling that fits, we will go back to the previous phoneme and try the next one in the list, until all the phonemes are successfully fitted onto the word. This recursive method is called backtracking, and is widely used on puzzle solving functions like a Sudoku solver.

In [7]:
def matchPhoneme(word):
    matchList = []
    phonemeList = dictionary[word][0]
    matchPhonemeKernel(word, phonemeList, matchList, 0)
    return matchList

def matchPhonemeKernel(word, phonemeList, matchList, index):
    if(index == len(phonemeList)): # base case
        if(len(word) != 0):
            return False
        return True
    phoneme = phonemeList[index]
    phonemeWithoutStress = phoneme
    if(len(phonemeWithoutStress) == 3):
        phonemeWithoutStress = phonemeWithoutStress[:2]
    for spelling in phonemeDict[phonemeWithoutStress]:
        if(spelling == word[0:len(spelling)]):
#             print(word)
            matchList.append((phoneme, spelling))
            subWord = word[len(spelling):]
            # recursive call
            success = matchPhonemeKernel(subWord, phonemeList, matchList, index + 1) 
            if(success): # if succeed then proceed
                return True
            # delete the failed attempt and try again in the next iteration
            del matchList[len(matchList) - 1] 
    return False

for word in exampleList:
    print(word)
    print(matchPhoneme(word))

fastidious
[('F', 'f'), ('AE0', 'a'), ('S', 's'), ('T', 't'), ('IH1', 'i'), ('D', 'd'), ('IY0', 'i'), ('AH0', 'ou'), ('S', 's')]
liability
[('L', 'l'), ('AY2', 'i'), ('AH0', 'a'), ('B', 'b'), ('IH1', 'i'), ('L', 'l'), ('IH0', 'i'), ('T', 't'), ('IY0', 'y')]
automatic
[('AO2', 'au'), ('T', 't'), ('AH0', 'o'), ('M', 'm'), ('AE1', 'a'), ('T', 't'), ('IH0', 'i'), ('K', 'c')]
thanks
[('TH', 'th'), ('AE1', 'a'), ('NG', 'n'), ('K', 'k'), ('S', 's')]
trace
[('T', 't'), ('R', 'r'), ('EY1', 'a'), ('S', 'ce')]
finish
[('F', 'f'), ('IH1', 'i'), ('N', 'n'), ('IH0', 'i'), ('SH', 'sh')]
decline
[('D', 'd'), ('IH0', 'e'), ('K', 'c'), ('L', 'l'), ('AY1', 'i'), ('N', 'ne')]
forbid
[('F', 'f'), ('ER0', 'or'), ('B', 'b'), ('IH1', 'i'), ('D', 'd')]
injection
[('IH0', 'i'), ('N', 'n'), ('JH', 'j'), ('EH1', 'e'), ('K', 'c'), ('SH', 'ti'), ('AH0', 'o'), ('N', 'n')]
difficulty
[('D', 'd'), ('IH1', 'i'), ('F', 'ff'), ('AH0', 'i'), ('K', 'c'), ('AH0', 'u'), ('L', 'l'), ('T', 't'), ('IY0', 'y')]
strike
[('S', 's'

### The last step: Separate syllables in spelling by merging the phonemic grouping and the matching

Now we have the grouping on phonemes and the matching from phonemes to spelling snippets, so the problem gets simple. We just need to simply substitute all the phonemes with their corresponding spelling snippets, and concatenate them. For different groups, we connect them with "-" to represent syllables. Then finally we will get a human-readable form like "liability: li-a-bi-li-ty".

In [14]:
def groupSpelling(word):
    result = ''
    phonemes = dictionary[word][0]
    grouped = groupPhonemes(phonemes)
    matchList = matchPhoneme(word)
    if(len(matchList) == 0):
        return 'ERROR: Empty matchlist'
    index = 0
    for group in grouped:
        for phoneme in group:
            result += matchList[index][1]
            index += 1
        result += '-'
    result = result[:len(result) - 1]
    return result

for word in exampleList:
    if(len(word) <= 7):
        print(word, end = '\t\t')
    else:
        print(word, end = '\t')
    print(groupSpelling(word))
    
print('\n')
    
while(True):
    print('Please enter another word to test the program. Enter "Q" to quit:')
    word = input()
    if(word == 'Q'):
        break
    print(dictionary[word][0])
    print(groupSpelling(word))
    count = countSyllables(dictionary[word][0])
    if(count == 1):
        print(count, end = ' syllable\n\n')
    else:
        print(count, end = ' syllables\n\n')

fastidious	fa-sti-di-ous
liability	li-a-bi-li-ty
automatic	au-to-ma-tic
thanks		thanks
trace		trace
finish		fi-nish
decline		de-cline
forbid		for-bid
injection	in-jec-tion
difficulty	di-ffi-cul-ty
strike		strike
swim		swim
construct	con-struct
eyebrow		eye-brow
ghastly		ghast-ly


Please enter another word to test the program. Enter "Q" to quit:
career
['K', 'ER0', 'IH1', 'R']
ERROR: Empty matchlist
2 syllables

Please enter another word to test the program. Enter "Q" to quit:
Q


### Conclusion

In this project, we start from counting syllables by the number of nucleus(vowels), and proceed to grouping phonemes into corresponding syllables. In the grouping process, we realized several principles in syllabification including maximal onset principle and the handling of onsetless syllables. By running a test we can prove the validity of the grouping by comparing the number of grouped syllables with the initial counting from nucleus. Then, by constructing a dictionary that matches phonemes to spelling snippets, we are able to run a backtracking algorithm that matches each phoneme with its corresponding spelling. Finally, with a simple substitution, we are able to break up the spelling into syllables in the way we break up phonemes.

Since we approach this problem from the IPA form, it dramatically increases the program's reliability and accuracy. The only insufficiencies of this program is that it relies on the installation of the NLTK corpus and can only process existing English words. In order to increase its robustness, we also need to further add to the dictionary so that it accounts for more irregular spelling patterns. Since we use a try-match approach, adding more entries to the dictionary will have little effect on the determinism of the program. 